### total 12,447개

미국 - 1042개(뉴스)  
미국 - 1094개(블로그)  

여행 - 991개(뉴스)  
여행 - 1098개(블로그)  

네이버 - 815개(뉴스)  
네이버 - 1944개(블로그)  

네이버블로그 - 453개(뉴스)  
네이버블로그 - 975개(블로그)  

네이버쇼핑 - 300개(뉴스)  
네이버쇼핑 - 1000개(블로그)  

네이버지도 - 508개(뉴스)  
네이버지도 - 924개(블로그)  

네이버클라우드 - 376개(뉴스)  
네이버클라우드 - 927개(블로그)  

***import***

In [1]:
import numpy as np
import pandas as pd
import pykospacing
spacer = pykospacing.Spacing()
from konlpy.tag import Okt
okt = Okt()
import re
from gensim import corpora
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

**데이터 불러오기**

In [2]:
NAVERBLOG_news = pd.read_csv('네이버블로그(뉴스)-453row.csv')
NAVERBLOG_blog = pd.read_csv('네이버블로그(블로그)-975row.csv')

In [3]:
NAVERBLOG_news

,title,link,description
0,"‘네카오’ 고리 풀렸다...네이버, AI기술력·실적 앞서",https://n.news.naver.com/mnews/article/016/000...,\n\n\n\n\n\n인공지능(AI) 광풍에 따라 해외 빅테크 기업이 일제히 상승세...
1,광고로 물든 '녹색창'… 점유율 하락중인 네이버[양철민의 아알못],https://n.news.naver.com/mnews/article/011/000...,\n\n네이버에 '우크라이나' 키워드 검색시맥락없는 '결혼광고'가 최상단에 노출반년...
2,서울시 경보문자보다 네이버 장애가 더 떨렸다[사이다IT],https://n.news.naver.com/mnews/article/003/001...,\n\n'뜬금' 없는 재난문자보다 네이버 앱 장애에 공포감플랫폼 의존사회 심화…포털...
3,큐시트·숏폼 제작도 AI가 척척…중소상공인 돕는 네이버 쇼핑라이브,https://n.news.naver.com/mnews/article/008/000...,\n\n\n\n\n\nAI가 써준 쇼핑라이브 큐시트. /사진=네이버네이버(NAVER...
4,네이버 광고 조작해 212억 벌어…광고 순위 조작단 기소,https://n.news.naver.com/mnews/article/055/000...,\n\n\n\n\n\n▲ 계정 판매업자로부터 압수한 다량의 휴대폰 등네이버에 특정 ...
...,...,...,...
448,"부산 동래구, 소상공인 역량 강화교육 수강생 모집",https://n.news.naver.com/mnews/article/003/001...,\n\n\n\n\n\n[부산=뉴시스] 부산 동래구청 임시청사. (사진=동래구 제공)...
449,"유튜브 공세가 매섭다…카톡, 지금 속도라면 하반기 추월당해",https://n.news.naver.com/mnews/article/028/000...,\n\n치솟는 유튜브·구글 점유율…OTT는 넷플릭스 독주\n\n\n\n유튜브·넷플릭...
450,"24시간 영업 하나로마트 양재점, 더위에 야간쇼핑 할인",https://n.news.naver.com/mnews/article/277/000...,"\n\n야식거리, 제철과일·채소, 정육 등밤 10시 이후 최대 50% 할인 시작농협..."
451,"성남시 청년프로예술팀, 9월까지 ‘버스킹’",https://n.news.naver.com/mnews/article/119/000...,\n\n26일 판교역 광장서 댄스 버스킹… 보컬팀·댄스팀 총 11회 공연\n\n\n...


### 전처리 함수 
*6주차 01.WordCloud 실습코드 참고*

- <span style = 'font-size:1.1em;line-height:1.5em'>(1) 줄바꿈 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(2) 오탈자 교정 (생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(3) 띄어쓰기 교정</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(4) sentence tokenization (너무 오래걸려서 생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(5) 특수문자 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(6) word tokenization (형태소 분석 및 stemming)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(7) stopword removal</span>

**불용어 목록 생성**

In [4]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거

In [5]:
def preprocessing(x):
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    results = []
    result = []
    x = x.replace('\n',' ') # (1) 줄바꿈 문자 제거
    df_news = spacer(x) # (3) 띄어쓰기 교정
    
    tokens = okt.morphs(x, stem=True) # (6) word tokenization
    for token in tokens:
        token = p.sub('', token) # (5) 특수문자 제거
        if token not in stopwords:
            result.append(token) # (7) stopword removal
    results.extend(result)
    results = ' '.join(results)
    return results

### pos_taging 함수

#(1) 품사 태깅 수행 

#(2) 명사, 형용사, 부사에 해당하는 단어들만 추출하여 저장

In [6]:
def pos_taging(x):
    pos_news = okt.pos(x) # (1)
    filtered_result = [word for word, pos in pos_news if pos in 'Noun'] # (2)
    nouns_news = [n for n in filtered_result if len(n) > 1]
    return nouns_news

### 전처리 및 postag

In [7]:
NAVERBLOG_news['preprocessed_description'] = NAVERBLOG_news['description'].apply(lambda x: preprocessing(x))
NAVERBLOG_news['preprocessed_description'] = NAVERBLOG_news['preprocessed_description'].apply(lambda x: pos_taging(x))

In [21]:
NAVERBLOG_blog['preprocessed_description'] = NAVERBLOG_blog['description'].apply(lambda x: preprocessing(x))
NAVERBLOG_blog['preprocessed_description'] = NAVERBLOG_blog['preprocessed_description'].apply(lambda x: pos_taging(x))

In [22]:
NAVERBLOG_news['preprocessed_description']

0      [인공, 지능, 광풍, 해외, 테크, 기업, 일제, 상승세, 보이, 국내, 대표, ...
1      [네이버, 우크라이나, 워드, 검색, 맥락, 결혼, 광고, 최상, 노출, 반년, 사...
2      [뜬금, 재난, 문자, 네이버, 장애, 공포, 플랫폼, 의존, 사회, 심화, 포털,...
3      [쇼핑, 라이브, 큐시트, 사진, 네이버, 네이버, 쇼핑, 라이브, 최근, 초대, ...
4      [계정, 판매업, 자로, 압수, 다량, 휴대폰, 네이버, 특정, 광고, 노출, 매크...
                             ...                        
448    [부산, 뉴시스, 부산, 동래, 구청, 임시, 청사, 사진, 동래구, 제공, 재판매...
449    [유튜브, 구글, 점유, 넷플릭스, 독주, 유튜브, 넷플릭스, 대표, 국외, 테크,...
450    [야식, 거리, 제철, 과일, 채소, 정육, 등밤, 이후, 최대, 할인, 시작, 농...
451    [판교역, 광장, 댄스, 버스킹, 보컬, 댄스, 공연, 성남시, 청년, 프로, 예술...
452    [네이버, 바이브, 제공, 네이버, 바이브, 통해, 지난, 공개, 오디오, 무비, ...
Name: preprocessed_description, Length: 453, dtype: object

### BOW(Bag-of-Words) 생성

In [23]:
word_dict_news = corpora.Dictionary(NAVERBLOG_news['preprocessed_description'])

In [24]:
word_dict_blog = corpora.Dictionary(NAVERBLOG_blog['preprocessed_description'])

In [25]:
corpus_news = [word_dict_news.doc2bow(text) for text in NAVERBLOG_news['preprocessed_description']]

In [26]:
corpus_blog = [word_dict_blog.doc2bow(text) for text in NAVERBLOG_blog['preprocessed_description']]

In [27]:
corpus_news[2]

[(14, 3),
 (17, 1),
 (18, 1),
 (22, 1),
 (24, 1),
 (27, 2),
 (30, 2),
 (33, 1),
 (34, 17),
 (39, 1),
 (40, 1),
 (41, 3),
 (42, 1),
 (43, 1),
 (64, 1),
 (67, 1),
 (75, 7),
 (77, 8),
 (80, 1),
 (93, 1),
 (106, 1),
 (119, 3),
 (120, 2),
 (127, 1),
 (132, 1),
 (134, 10),
 (142, 1),
 (148, 3),
 (168, 2),
 (174, 1),
 (177, 1),
 (181, 5),
 (183, 3),
 (191, 3),
 (202, 2),
 (211, 1),
 (213, 1),
 (222, 1),
 (223, 1),
 (232, 1),
 (236, 2),
 (249, 1),
 (261, 4),
 (264, 1),
 (272, 1),
 (273, 1),
 (277, 1),
 (282, 7),
 (289, 1),
 (296, 6),
 (303, 1),
 (305, 1),
 (306, 2),
 (322, 1),
 (323, 1),
 (327, 1),
 (346, 1),
 (347, 1),
 (348, 2),
 (349, 1),
 (350, 2),
 (351, 1),
 (352, 2),
 (353, 1),
 (354, 3),
 (355, 4),
 (356, 1),
 (357, 1),
 (358, 4),
 (359, 2),
 (360, 1),
 (361, 1),
 (362, 1),
 (363, 5),
 (364, 1),
 (365, 1),
 (366, 1),
 (367, 1),
 (368, 1),
 (369, 1),
 (370, 1),
 (371, 1),
 (372, 1),
 (373, 1),
 (374, 1),
 (375, 1),
 (376, 1),
 (377, 1),
 (378, 1),
 (379, 2),
 (380, 3),
 (381, 1),
 (382,

In [28]:
[(idx,word) for idx, word in word_dict_news.items()]

[(0, '가변'),
 (1, '가운데'),
 (2, '가의'),
 (3, '가이드'),
 (4, '각기'),
 (5, '갈리'),
 (6, '감소'),
 (7, '강점'),
 (8, '개년'),
 (9, '개인'),
 (10, '거대'),
 (11, '거래'),
 (12, '거래소'),
 (13, '건립'),
 (14, '검색'),
 (15, '견조'),
 (16, '결과물'),
 (17, '경우'),
 (18, '계획'),
 (19, '고객'),
 (20, '고도화'),
 (21, '공개'),
 (22, '관련'),
 (23, '광풍'),
 (24, '국내'),
 (25, '권제'),
 (26, '규모'),
 (27, '기간'),
 (28, '기록'),
 (29, '기술'),
 (30, '기업'),
 (31, '기업인'),
 (32, '기여'),
 (33, '기자'),
 (34, '네이버'),
 (35, '다중화'),
 (36, '대규모'),
 (37, '대비'),
 (38, '대신'),
 (39, '대응'),
 (40, '대표'),
 (41, '대한'),
 (42, '대해'),
 (43, '데이터'),
 (44, '동기'),
 (45, '라며'),
 (46, '리다'),
 (47, '리츠'),
 (48, '리포트'),
 (49, '매년'),
 (50, '매도'),
 (51, '매수'),
 (52, '매출'),
 (53, '모델'),
 (54, '모멘텀'),
 (55, '목표'),
 (56, '반면'),
 (57, '발표'),
 (58, '발휘'),
 (59, '방침'),
 (60, '방향성'),
 (61, '배당'),
 (62, '벤치마크'),
 (63, '보유'),
 (64, '보이'),
 (65, '부문'),
 (66, '분야'),
 (67, '블로그'),
 (68, '비즈'),
 (69, '사주'),
 (70, '삼성'),
 (71, '상반'),
 (72, '상승'),
 (73, '상승세'),
 (74, '상향'),
 (75, '상황'),
 (76,

In [29]:
[(idx,word) for idx, word in word_dict_blog.items()]

[(0, '가격'),
 (1, '가운데'),
 (2, '가장'),
 (3, '감사'),
 (4, '강동구'),
 (5, '강동대로'),
 (6, '강조'),
 (7, '걱정'),
 (8, '검색'),
 (9, '결과'),
 (10, '결정'),
 (11, '경우'),
 (12, '경험'),
 (13, '계시'),
 (14, '계약'),
 (15, '고민'),
 (16, '과정'),
 (17, '관련'),
 (18, '광고'),
 (19, '구매'),
 (20, '그것'),
 (21, '깨달'),
 (22, '나니'),
 (23, '나볼'),
 (24, '내기'),
 (25, '네이버'),
 (26, '노출'),
 (27, '누군가'),
 (28, '다시'),
 (29, '단연'),
 (30, '대표'),
 (31, '대한'),
 (32, '대해'),
 (33, '도움'),
 (34, '때문'),
 (35, '리다'),
 (36, '마음'),
 (37, '마진'),
 (38, '마찬가지'),
 (39, '마케팅'),
 (40, '만난'),
 (41, '만날'),
 (42, '만족'),
 (43, '말씀'),
 (44, '매출'),
 (45, '무언가'),
 (46, '문의'),
 (47, '발전'),
 (48, '보시'),
 (49, '보장'),
 (50, '부담'),
 (51, '부분'),
 (52, '분야'),
 (53, '불구'),
 (54, '브랜드'),
 (55, '블로그'),
 (56, '비교'),
 (57, '비용'),
 (58, '사람'),
 (59, '사실'),
 (60, '사업'),
 (61, '사이트'),
 (62, '상위'),
 (63, '생각'),
 (64, '서울특별시'),
 (65, '선택'),
 (66, '소비자'),
 (67, '소통'),
 (68, '솔루션'),
 (69, '수도'),
 (70, '수익'),
 (71, '스트레스'),
 (72, '시대'),
 (73, '시작'),
 (74, '신뢰'),
 (75, '실력'),
 (

In [30]:
len(word_dict_news)

7063

### LDA 모델 훈련

#(1) 3개의 topic

#(2) LDA 모델을 생성. 'id2word'에는 단어 사전을 전달하고, 학습 반복 횟수는 15로 설정

#(3) 생성된 모델에서 각 주제에 대한 상위 4개 단어를 출력


In [39]:
N_TOPICS = 3 # (1) 
ldamodel_news = gensim.models.ldamodel.LdaModel(corpus_news, num_topics = N_TOPICS, id2word=word_dict_news, passes = 15) # (2)

topics = ldamodel_news.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.030*"블로그" + 0.023*"네이버" + 0.014*"사용자" + 0.009*"세대"')
(1, '0.040*"네이버" + 0.024*"서비스" + 0.013*"검색" + 0.009*"블로그"')
(2, '0.033*"중국" + 0.032*"네이버" + 0.020*"접속" + 0.012*"차단"')


In [40]:
N_TOPICS = 3 # (1)
ldamodel_blog = gensim.models.ldamodel.LdaModel(corpus_blog, num_topics = N_TOPICS, id2word=word_dict_blog, passes = 15) # (2)

topics = ldamodel_blog.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.055*"블로그" + 0.027*"네이버" + 0.012*"설정" + 0.011*"이미지"')
(1, '0.068*"블로그" + 0.031*"네이버" + 0.020*"수익" + 0.011*"포스팅"')
(2, '0.019*"블로그" + 0.013*"네이버" + 0.012*"이웃" + 0.011*"생각"')


### LDA 시각화

In [41]:
vis_news = pyLDAvis.gensim_models.prepare(ldamodel_news, corpus_news, word_dict_news)
pyLDAvis.display(vis_news)

In [42]:
vis_blog = pyLDAvis.gensim_models.prepare(ldamodel_blog, corpus_blog, word_dict_blog)
pyLDAvis.display(vis_blog)